# Optimizing data access and processing

Over the multiple formats presented in the other tutorials, some were presented as cloud optimized. This notebook shows some methods to take advantage of those formats to optimize data access and processing. The methods presented are specifically optimized for cloud usage but can (and should most of the time) also be used with local files. The general idea is to limit the data access: either by loading only the required data or opening small fractions (chunks) of a whole dataset and iterate over all these chunks.


## Data used
This tutorial assumes the user ran the other notebooks before, or at least the corresponding ones, e.g. to run the *raster* section of this notebook, it is recommended to run the [raster formats tutorial](./raster_formats.ipynb) first. They give context on the data used and basic instructions on how to deal with the different types and formats. Moreover, the other tutorials download or generate sample data used as example, so make sure you at least ran the generation parts of the corresponding tutorials, or use your own data.

## Raster data

In the [raster formats tutorial](./raster_formats.ipynb), COGs (Cloud Optimized Geotiffs) were presented as the best cloud-oriented solution, as they are natively split and compressed in chunks and allow overviews. In this section, we present how to load and process a raster chunk by chunk. This is particularly useful when using large datasets which can't be loaded in memory (both on local or cloud architectures) or when accessing a specific sub area of a temporal series of images.

### Using rasterio

Reading chunks with rasterio is done using a window: a smaller section of the image, defined by an offset and a size on the x and y axes.


A simple way to implement reading an image chunk by chunk is using chunks with the same width as the image (i.e. one or multiple lines) and iterating over them.

In [ ]:
import rasterio
from rasterio.windows import Window

cog_path = "./sample_data/rasters/data/xt_SENTINEL2B_20180621-111349-432_L2A_T30TWT_D_V1-8_RVBPIR_cog.tif"

In [ ]:
# read the entire raster (for comparison)
with rasterio.open(cog_path) as src:
    %time data = src.read(1)

In [ ]:
# read chunk by chunk
x_chunk_size = 1000  # number of columns per chunk
y_chunk_size = 1000  # number of rows per chunk

with rasterio.open(cog_path) as src:
    width = src.width
    height = src.height
    for start_row in range(0, height, y_chunk_size):  # iterate over rows
        end_row = min(start_row + y_chunk_size, height)
        for start_col in range(0, width, x_chunk_size):  # iterate over columns
            end_col = min(start_col + x_chunk_size, width)
            window = Window(start_col, start_row, end_col - start_col, end_row - start_row)
            %time data = src.read(1, window=window)  # read the 1st band
            print(data.shape)

But there is an easier and more optimized way of dealing with rasters by chunks. COGs have an intern chunk size used when writing them. It is easier and usually more efficient to use this chunk size instead, using `src.block_windows`. In this code `ji` is the current chunk's coordinates, relative to the chunk grid. The coordinates of the first pixel of the chunk are given by the widow's column and row offset.

In [ ]:
with rasterio.open(cog_path) as src:
    for ji, window in src.block_windows(1):
        print(f"Chunk's coordinates: {ji}")
        print(window)
        data = src.read(window=window)
        print(data.shape, "\n")

### Using rioxarray

Rasterio opens raster as numpy arrays and processes them sequentially. Instead, it is possible to use `xioxarray`, a library to open rasters as xarrays:

In [ ]:
import rioxarray as rxr

xds = rxr.open_rasterio(cog_path, chunks=True)
xds

The `rioxarray` [official documentation](https://corteva.github.io/rioxarray/stable/examples/read-locks.html#Chunking) details the chunking process and how to optimize chunk management. This also allows the use of `dask` to process chunks in parallel. Once again, the official documentation has details on how to use dask with rioxarray: https://corteva.github.io/rioxarray/stable/examples/dask_read_write.html.

## Vector data

Vector data allows two types of optimizations:

- streaming data chunk by chunk (similar to raster files)
- filtering input data before loading it in memory

These optimizations are not necesarily available for all vector formats. Geoparquet is the most suitable format for these uses: it can be easily streamed and filtered. 

### Streaming

First, let's try reading data chunk by chunk. This can be easily done using `pyarrow`. To illustrate this example, we'll re-use the `landuse.parquet` file generated in the [vector formats tutorial](./vector_data_formats.ipynb) and count the number of forest polygons in each chunk (or batch).

In [ ]:
import pyarrow.parquet as pq
file_path = "./sample_data/vector/departement-31/landuse.parquet"

parquet_file = pq.ParquetFile(file_path)

for n, batch in enumerate(parquet_file.iter_batches(batch_size=1000)):  # batch_size: number of rows to read
    print(f"Batch number {n}, n_rows: {batch.num_rows}")
    column=batch.column("type").to_pandas()  # select the column 'type' and transform the column to a pandas series
    n_forest = len(column[column == "forest"].index)  # count the occurrences of 'forest' 
    print(f"{n_forest} occurences of 'forest'\n")

## Datacube

Raster (and, in a less significant way vector) formats can be optimized for cloud computing, but may require some non-trivial work. However, most datacube formats are natively cloud-optimized (or can easily be optimized for cluod usage), like zarr. Let's reuse the `.zarr` file generated in the [data cube formats tutorial](./datacube_formats.ipynb). As explained in that tutorial, `.zarr` files are natively split into chunks to facilitate chunking mechanisms. Another optimization is lazy loading and computing. Lazy loading means that the data is not loaded into memory until needed; only the metadata is stored. This allows the user to define operations that, similarly, are not computed until required. The loading and computing steps are then optimized and executed at the end, improving ressource usage. Here is an example:

In [ ]:
import hvplot.xarray
import xarray as xr
zarr_path = "./sample_data/data_cube/example_from_xarray.zarr"

da = xr.open_dataset(zarr_path)["data"]
da.variable

The actual values of the array aren't loaded. One way of explicitly loading them into memory is using the `.load()` method:

In [ ]:
da.load()
da.variable

Let's reopen it to reset it to lazy loaded and test lazy computing. For this, let's multiply the array by 10 and compute its mean over all time periods:

In [ ]:
da = xr.open_dataset(zarr_path, chunks="auto", engine="zarr")["data"]

print("Lazy loaded array:")  # lazy loaded
print(da, "\n\n")

scaled_data = da * 10
mean_data = scaled_data.mean(dim="time")  # lazy computed
print("Lazy computed array:")
print(mean_data, "\n\n")


result = mean_data.compute()  # load in memory and compute
print("Loaded and computed:")
print(result)

As explained in the rasters section, an efficient optimization method is to run chunks computation in parallel, using dask for example. And `xarray` uses dask to handle data arrays, making it easier to assemble these methods.

## Conclusion

There are many optimization techniques available for all the data types and formats presented, usable both on local and cloud architectures. It is important to consider which format and what processing methods can and should be used, depending on the architecture and ressources available, as well as the data volume. Generally, a modern and optimized code should be considered, but over a  small dataset, using parallel processing and chunking data may introduce unnecessary complexity and might be inefficient.